In [1]:
import pymysql

In [2]:
conn = pymysql.connect(host='localhost', user='root', password=str(input()), db='sales') 
curs = conn.cursor(pymysql.cursors.DictCursor)

In [6]:
sql_p_data = '''
      WITH p_data AS (
            SELECT *
            FROM online_sales
            WHERE CustomerID IS NOT NULL AND
                  Quantity > 0 AND
                  UnitPrice > 0
            GROUP BY 1,2,3,4,5,6,7,8
      )
      ''' 

sql_first_purchase = '''
      WITH first_purchase AS (
            SELECT CUSTOMER_ID, DATE(MIN(InvoiceDate)) AS CohortDate
            FROM p_data
            GROUP BY CUSTOMER_ID
      )
      '''

sql_cohort_group = '''
      WITH cohort_grouped AS (
      SELECT d.CustomerID, f.CohortDate, DATE_DIFF(DATE(InvoiceDate), CohortDay, MONTH) AS CohortIndex, MONTH(f.CohortDate) AS CohortGroup
      FROM p_data d LEFT JOIN first_purchase f
      ON s.CustomerID = f.CustomerID
      )
'''

sql_cohort_anaysis = '''
      SELECT CohortGroup, CohortIndex, COUNT(DISTINCT CustomerID) AS CustomerCount
      FROM cohort_grouped
      GROUP BY 1,2
'''

In [7]:
curs.execute(sql_p_data+','+sql_first_purchase+','+sql_cohort_group+sql_cohort_anaysis)

ProgrammingError: (1064, "You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near 'WITH first_purchase AS (\n            SELECT CUSTOMER_ID, MONTH(MIN(InvoiceDate))' at line 10")